# Workshop 10: Approximate Root-Finding Methods

**Submit this notebook to bCourses to receive a grade for this Workshop.**

Please complete workshop activities in code cells in this iPython notebook. The activities titled **Practice** are purely for you to explore Python. Some of them may have some code written, and you should try to modify it in different ways to understand how it works. Although no particular output is expected at submission time, it is _highly_ recommended that you read and work through the practice activities before or alongside the exercises. However, the activities titled **Exercise** have specific tasks and specific outputs expected. Include comments in your code when necessary. Enter your name in the cell at the top of the notebook. The workshop should be submitted on bCourses under the Assignments tab.

In lecture, you discussed a few different ways to estimate roots of nonlinear functions of one variable. Here we will expand on the details and use some of those techniques.

## Root finding

### Iterative Relaxation Method

A fixed point of a function $g$ is defined to be a point $x_0$ where $g(x_0) = x_0$. One method of finding a fixed point is to simply guess a value, and keep applying $g$ to it. If at some iteration we reach the fixed point, then every iteration after it will return the same value, and we know we have solved the equation. So we can try applying $g$ for many iterations and watch for convergence. Doing so solves the equation

$$g(x) = x$$

But what we want to solve is, for a function $f(x)$, 

$$f(x) = 0$$

So this method works automatically if the function $f(x)$ whose root we want to find contains a term $-x$ in it, such as

$$f(x) = 2 - e^{-x} - x$$

Then we can use the method above to solve

$$g(x) = 2 - e^{-x} = x$$

But we can also try to solve functions which don't contain $-x$ by simply adding $x$ to both sides of the root equation:

$$f(x) + x = x$$

So $g(x) = f(x) + x$.

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

def fixedpt_finding(func, x0, nIter=50):
    '''
        fixedpt_finding uses an iterative relaxation method to solve the problem func(x) == x
        
        Inputs:
            func - name of function. func must be a python function which
                   takes only required argument and returns one value
            
            x0 - initial guess for the location of the fixed point
            nIter - number of iterations. Default value is 50.
            
        Outputs:
            x - the final estimate of the location of the fixed point
            prec - an estimate of the precision of the location, evaluated 
                as the difference between the last and second last estimates
                of the location of the fixed point
            xa - numpy array of the nIter estimates of the location of the fixed point
    '''
    xa = []
    x = x0
    for i in range(nIter):
        xa.append(x)
        x = func(x)
        
    prec = xa[-1]-xa[-2]
    
    return x, prec, np.array(xa)

In [ ]:
# Example

def f(x):
    return np.cos(x)**2 - 0.5

def g(x):
    return f(x) + x

x_root, precision, _ = fixedpt_finding(g, 0.0, nIter=50)

print("Root:\t\tx= ", x_root)
print("Precision:\tdx=", precision)

xd = np.linspace(-4,4,100)

# Plot f(x) and see if the estimated root is in the right place
plt.figure()
plt.plot(xd, f(xd),label='f(x)')
plt.plot(xd, np.zeros(len(xd)),label='y=0')
plt.plot(x_root, f(x_root),marker='o',markersize=10, label='root found')
plt.legend()

# Plot g(x) and visualize the fixed point
plt.figure()
plt.plot(xd, g(xd),label='g(x)')
plt.plot(xd, xd,label='y=x')
plt.plot(x_root, g(x_root),marker='o',markersize=10, label='fixed pt')
plt.legend()

plt.show()

## Exercise 1
a. Use the iterative relaxation method to find the root of $f(x) = \cos(x)-x$. Print your solution and precision. Make a plot of the guessed value of the root vs. iteration number to demonstrate the convergence to the solution. Feel free to use the function defined above.

b. Now use the iterative relaxation method to find a root of $f(x) = \cos(x)$. Print your solution and precision. Plot the value vs. iteration number. Compare your answer to the analytical value.

c. Now find the root of $f(x) = e^{x}-x-2$. Print your solution and precision, and plot the value vs. iteration number. Try a few different guesses and numbers of iterations. How many roots does this function have? How many were you able to find using this method?

d. Lastly, use this method to find the root of $f(x) = \cos(\frac{\pi x}{2})-x$. Print your solution and precision. Plot the value vs. iteration number. Does the method converge to a solution? Does a root exist for this function?

In [ ]:
# Code for Exercise 1

### Root finding using the bisection method

First we introduce the `bisect` algorithm which is (i) robust and (ii) slow but conceptually very simple.

Suppose we need to compute the roots of 

$$f(x) = x^3 - 2x^2$$

This function has a roots at $x=0, 2$. Run the cell below to generate a plot of $f(x)$. What do you notice about how the function behaves around each of these two zeros?

In [ ]:
def f(x):
    return x ** 3 - 2 * x ** 2

# Visualize f(x) and see the roots
xd = np.linspace(-1,3,100)
yd = f(xd)
plt.figure()
plt.plot(xd,yd, label='f(x)')
plt.plot(xd,np.zeros(len(xd)))
plt.legend()
plt.show()

Indeed, the `bisect` method operates on the "Intermediate Value Theorem" which just makes the observation that if a continuous function $f(x)$ changes sign over an interval $x\in [a,b]$, then there must exist at least one value of $x$ in that interval for which $f(x) = 0$. As a result, this method cannot find the root at $x=0$. So to use `scipy.optimize.bisect` you need to give it three arguments: the name of the python function which encodes $f(x)$, the left end of your interval ($a$) and the right end of your interval $b$. But further more, $a$ and $b$ must be such that $f(a)$ and $f(b)$ have opposite sign. Try changing some of these values in the call to `bisect` below!

In [ ]:
from scipy.optimize import bisect

x_root = bisect(f, 1.5, 3, xtol=1e-6) #xtol is an optional argument specifying how precise we want the answer

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))

## Exercise 2
a. Use the built-in `bisect` function to compute the roots of the four functions from Exercise 2, and compare the results. Was the bisect method able to find all of the roots of $f(x) = e^{x} - x - 2$? What about for $f(x) = \cos(\frac{\pi x}{2})-x$?


In [ ]:
# Code for Exercise 2

### Root finding using Brent method

This is another method to find a root of the function $f(x)$ on the sign changing interval $x\in[a , b]$. It is a safe version of the secant method that uses inverse quadratic extrapolation. Brent’s method combines a few other elementary root-finding techniques: root bracketing, interval bisection, and inverse quadratic interpolation. But again, this still requires that $f(a)$ and $f(b)$ have opposite signs.

In [ ]:
from scipy.optimize import brentq

def f(x):
    return x ** 3 - 2 * x ** 2

x = brentq(f, 0.5,3, xtol=1e-6)

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))

### Root finding using the `fsolve` function

A (often) better (in the sense of “more efficient”) algorithm than the bisection algorithm is implemented in the general purpose `fsolve()` function for root finding of (multidimensional) functions. This algorithm needs only one starting point close to the suspected location of the root (but is not garanteed to converge).

Here is an example:

In [ ]:
from scipy.optimize import fsolve

def f(x):
    return x ** 3 - 2 * x ** 2

x = fsolve(f, [-0.5,3])           # look for two roots starting with 0 and 3

print("Number of roots is", len(x))
print("The root(s) are ", x)

The input to `fsolve` is the name of the python function and the array of initial locations for the roots you are trying to find. The return value of `fsolve` is a numpy array of the best estimates of the locations of the roots found for each initial guess given. If $n$ initial guesses are given, $n$ estimates are returned.

## Exercise 3 
(Newman 6.15)

Consider a sixth-order polynomial 
$$P(x) = 924x^6 - 2772x^5 + 3150x^4 -1680x^3 +420x^2 -42x + 1$$
There is no general formula for the roots of a polynomial of degree 6, but you can compute the roots numerically. 
1. Make a plot of $P(x)$ from $x=0$ to $x=1$ and by inspecting it find rough values for the six roots of the polynomial. 
1. Write the code to solve for the positions of all six roots to at least ten decimal places using at least one of the methods above (you can use the built-in functions).

In [ ]:
# Code for Exercise 3